# <font color='blue'> Cours Science de données - IFRISSE 2020 - PART3</font>

# Dans cette partie de notre étude, nous aborderons la notion de feature engineering et la construction de modèle de Machine Learning

# I - Feature Engineering - Extraction de caractéristiques

L'extraction de caractéristiques est considérée comme étant les méthodes permettant de sélectionner et/ou combinent des variables caractéristiques dans le but **1)** de réduire efficacement la quantité de données à traiter, et **2)** tout en décrivant de manière précise et complète l'ensemble de données d'origine (sans perdre le contenu principal des données).

### Qu'est-ce qu'une caractéristique et pourquoi avons-nous besoin d'une ingénierie ?

Fondamentalement, tous les algorithmes d'apprentissage machine utilisent des données données d'entrées pour créer/proposer des sorties. 

Ces données d'entrées comportent des caractéristiques, qui généralement se présentent  sous la forme de colonnes structurées. 

Pour fonctionner correctement, les algorithmes ont besoin de caractéristiques spécifiques (plus ou moins précises). C'est là que le besoin d'ingénierie des caractéristiques se fait sentir. Elles ont principalement deux objectifs :

**1° Préparer le jeu de données d'entrée approprié, compatible avec les exigences du modèle ou de l'algorithme d'apprentissage.**

**2° Améliorer les performances des modèles d'apprentissage.**

### Bon à Savoir : src => Source: https://www.forbes.com/sites/gilpress

![alt text](f_eng.jpg "Source: https://www.forbes.com/sites/gilpress")

# Quelques techniques utilisées pour l'extraction de caractéristiques 

1.Imputation => Imputation

2.Handling Outliers => le traitement des valeurs aberrantes

3.Binning => jumelage/échantilonnage

4.Log Transform => transformation logarithmic

5.One-Hot Encoding => codage à valeur binaire 0|1

6.Grouping Operations  => opérations de regroupement

7.Feature Split => Découpage 

8.Scaling => mise à l'échelle

#### NB: Certaines techniques pourraient être utilisées dans des cas spécifiques, tandis que d'autres pourraient être bénéfiques dans tous les cas.

In [56]:
# Les librairies de bases les plus utilisées 
import pandas as pd #  analysise de données/transformation
import numpy as np # manipulation de valeurs numeriques et tableaux

## 1 - Imputation => resoudre les problèmes de valeurs manquentes

Les valeurs manquantes sont l'un des problèmes les plus courants que vous pouvez rencontrer lorsque vous essayez de préparer vos données pour l'apprentissage. 

Les valeurs manquantes peuvent être dues à des erreurs humaines, des interruptions dans le flux (de collecte) de données, des problèmes de confidentialité, etc.

#### Solutions

### a° <font color='blue'>Supprimer les colonnes ou les lignes contenant des valeurs manquentes en fonction d'un seuil donné</font>

threshold = 0.7

#Dropping columns with missing value rate higher than threshold !attention car cela peut affecter fortement la distribution des données d'origines

data = data[data.columns[data.isnull().mean() < threshold]]

#Dropping rows with missing value rate higher than threshold

data = data.loc[data.isnull().mean(axis=1) < threshold]

### b° <font color='blue'>Remplacer par une valeur X lorsque les valeurs manquentes sont des valeurs numériques</font>

#Filling all missing values with 0

data = data.fillna(0)

#Filling missing values with medians of the columns

data = data.fillna(data.median())

#Filling missing values with std of the columns

data = data.fillna(data.std())


### c° <font color='blue'>Remplacer par la valeur la plus fréquente lorsque les valeurs manquentes sont des valeurs catégorielles</font>

#Max fill function for categorical columns

data['column_name'].fillna(data['column_name'].value_counts().idxmax(), inplace=True)

## 2.Handling Outliers => le traitement des valeurs aberrantes


Les valeurs aberantes peuvent être détecté de deux façons. Soit en partant de **1)** l'écart type, ou par  **2)** la technique des percentiles

#### Solutions

### a° <font color='blue'>la technique de la deviation</font>

Si une valeur a une distance par rapport à la moyenne supérieure à **x * écart type**, elle peut être considérée comme une valeur aberrante. Alors que devrait être **x** ?

Il n'y a pas de solution triviale pour **x**, mais généralement, une valeur comprise entre **2 et 4** semble pratique.

#### obtenir et supprimer les valeurs aberantes

x = 3

upper_lim = data['column'].mean () + data['column'].std () * x

lower_lim = data['column'].mean () - data['column'].std () * x

data = data[(data['column'] < upper_lim) & (data['column'] > lower_lim)]

## 3.Binning

Cette technique est utilisable que ce soit sur des variables numériques ou catégorielles. La principale motivation du binning est de rendre le modèle plus robuste et d'éviter les problèmes de sur-apprentissage.

### Exemple avec de variables Numéric
Value      Bin       
0-30   ->  Low       
31-70  ->  Mid       
71-100 ->  High
### Exemple avec de variables Categorielles
Value      Bin       
Spain  ->  Europe      
Italy  ->  Europe       
Chile  ->  South America
Brazil ->  South America

In [57]:
df_n =  pd.DataFrame([2,45,7,85,28], columns=['value'])
df_n

,value
0,2
1,45
2,7
3,85
4,28


In [58]:
#Exemple avec de variables Numéric

df_n['bin'] = pd.cut(df_n['value'], bins=[0,30,70,100], labels=["Low", "Mid", "High"])
df_n

,value,bin
0,2,Low
1,45,Mid
2,7,Low
3,85,High
4,28,Low


In [59]:
df_c =  pd.DataFrame(['Spain','Chile','Australia','Italy','Brazil', 'Burkina Faso'], columns=['Country'])
df_c

,Country
0,Spain
1,Chile
2,Australia
3,Italy
4,Brazil
5,Burkina Faso


In [60]:
#Exemple avec de variables Categorielles
conditions = [
    df_c['Country'].str.contains('Spain'),
    df_c['Country'].str.contains('Italy'),
    df_c['Country'].str.contains('Chile'),
    df_c['Country'].str.contains('Brazil'),
    df_c['Country'].str.contains('Burkina Faso')]

choices = ['Europe', 'Europe', 'South America', 'South America', 'Africa']

df_c['Countryx'] = np.select(conditions, choices, default='Other')
df_c

,Country,Countryx
0,Spain,Europe
1,Chile,South America
2,Australia,Other
3,Italy,Europe
4,Brazil,South America
5,Burkina Faso,Africa


## 4.Log Transform => transformation logarithmic

C'est l'une des transformations mathématiques les plus couramment utilisées dans l'ingénierie des caractéristiques.

Quels sont les avantages de la transformation logarithmique ?

##### Elle permet de traiter des données biaisées et, après transformation, la distribution devient plus proche de la normale.
##### Il diminue également l'effet des valeurs aberrantes, grâce à la normalisation des différences d'amplitude et permet d'avoir un modèle devient plus robuste.

In [61]:
#Log Transform Example
data = pd.DataFrame({'value':[2,45, -23, 85, 28, 2, 35, -12]})
data['log+1'] = (data['value']+1).transform(np.log)

#Negative Values Handling
#Note that the values are different
data['log'] = (data['value']-data['value'].min()+1) .transform(np.log)
data

,value,log+1,log
0,2,1.098612,3.258097
1,45,3.828641,4.234107
2,-23,NaN,0.000000
3,85,4.454347,4.691348
4,28,3.367296,3.951244
5,2,1.098612,3.258097
6,35,3.583519,4.077537
7,-12,NaN,2.484907


## 5. One-hot encoding 

Cette méthode répartit les valeurs d'une colonne sur plusieurs colonnes et leur attribue 0 ou 1. Ces valeurs binaires expriment la relation entre la colonne groupée et la colonne codée.

![alt text](one_hot.png 'one_hot encod')


## 7.Feature Split.


Cette techiq permet de rendre certaines variables plus compréhensible, favorisant la compréhension de l'algo au moment de l'apprentissage (moins d'ambiguité):

* Nous permettons aux algorithmes d'apprentissage de les comprendre.

* Rendre possible leur binning et leur regroupement.

* Améliorer les performances des modèles en découvrant des informations potentielles.

In [62]:
df_split = pd.DataFrame({'name':['Luther N. Gonzalez','Charles M. Young', 'Terry Lawson', 'Kristen White', 'Thomas Logsdon']})
df_split

,name
0,Luther N. Gonzalez
1,Charles M. Young
2,Terry Lawson
3,Kristen White
4,Thomas Logsdon


In [63]:
#Extraire le prenom
df_split.name.str.split(" ").map(lambda x: x[0])


0     Luther
1    Charles
2      Terry
3    Kristen
4     Thomas
Name: name, dtype: object

In [64]:
#Extraire le  nom
df_split.name.str.split(" ").map(lambda x: x[-1])

0    Gonzalez
1       Young
2      Lawson
3       White
4     Logsdon
Name: name, dtype: object

## 8. Scaling => mise à l'échelle

### Normalisation

Dans la plupart des cas, les variables n'ont pas une certaine harmonisation et elles diffèrent les unes des autres.



In [65]:
# exemple de normalisation =>
data = pd.DataFrame({'value':[2,45, -23, 85, 28, 2, 35, -12]})

data['normalized'] = (data['value'] - data['value'].min()) / (data['value'].max() - data['value'].min())
data

,value,normalized
0,2,0.231481
1,45,0.629630
2,-23,0.000000
3,85,1.000000
4,28,0.472222
5,2,0.231481
6,35,0.537037
7,-12,0.101852


### Standardization

La standardisation (ou normalisation du z-score) met à l'échelle les valeurs tout en tenant compte de l'écart type. 

Si l'écart-type des caractéristiques est différent, leur plage sera également différente. Cela permet de réduire l'effet des valeurs aberrantes des caractéristiques.



In [66]:
# xemple de standardisation
data = pd.DataFrame({'value':[2,45, -23, 85, 28, 2, 35, -12]})

data['standardized'] = (data['value'] - data['value'].mean()) / data['value'].std()
data

,value,standardized
0,2,-0.518878
1,45,0.703684
2,-23,-1.229670
3,85,1.840952
4,28,0.220346
5,2,-0.518878
6,35,0.419367
7,-12,-0.916922


# <font color='blue'> END OF FEATURE ENGINEERING</font>